<a href="https://colab.research.google.com/github/1Anass/AIRegsAgent/blob/main/AIReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load text in a variable
preprocessing


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install PyPDF2
!pip install python-docx
!pip install pycryptodome
!pip install nltk
!pip install contractions
!pip install sentence-transformers
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 85.8 MB/s eta 0:00:0

Function that reads content of a pdf file


In [ ]:
import os
from pathlib import Path
import PyPDF2
import docx

def read_pdf(file_path):
    content = {}  # Dictionary to store page number as key and paragraphs as values
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)

        # Check if the PDF is encrypted
        if pdf_reader.is_encrypted:
            try:
                pdf_reader.decrypt('')  # Attempt to decrypt with an empty password
            except PdfReadError:
                print(f"Could not decrypt {file_path}. Skipping file.")
                return None  # Skip this file if it cannot be decrypted

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            # Split page text into paragraphs (based on double newlines or some other delimiter)
            paragraphs = [para.strip() for para in page_text.split('\n\n') if para.strip()]
            content[page_num + 1] = paragraphs  # Page numbers are 1-based
    return content

Load files in following format { (doc_name : [(pagenumber, paragraph)]) }

In [ ]:
folder_path = '/content/drive/MyDrive/eu_ai_Act/'  # Adjust this path as necessary

documents = {}

# Loop through all files in the folder and read their content
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    if filename.endswith(".pdf"):
        print(f"Processing: {file_path}")

        # Check if the file is encrypted and try to decrypt
        pdf_reader = PyPDF2.PdfReader(file_path)
        if pdf_reader.is_encrypted:
            print(f"{filename} is encrypted, attempting to decrypt...")
            try:
                pdf_reader.decrypt('')  # Attempt to decrypt with an empty password
                documents[filename] = read_pdf(file_path)  # Proceed with reading after decryption
                print(f"Successfully decrypted and read: {filename}")
            except Exception as e:
                print(f"Could not decrypt {filename}. Skipping file. Error: {e}")
        else:
            # If not encrypted, just read the file
            documents[filename] = read_pdf(file_path)
            print(f"Successfully read: {filename}")

# Check the loaded documents
print("Documents successfully loaded:", documents.keys())

Processing: /content/drive/MyDrive/eu_ai_Act/EY_eu_ai_act.pdf
Successfully read: EY_eu_ai_act.pdf
Processing: /content/drive/MyDrive/eu_ai_Act/Official_eu_ai_act.pdf
Successfully read: Official_eu_ai_act.pdf
Processing: /content/drive/MyDrive/eu_ai_Act/KPMG_eu_ai_act.pdf
KPMG_eu_ai_act.pdf is encrypted, attempting to decrypt...
Successfully decrypted and read: KPMG_eu_ai_act.pdf
Processing: /content/drive/MyDrive/eu_ai_Act/2024.01.24-Communication-Start-Ups.pdf
Successfully read: 2024.01.24-Communication-Start-Ups.pdf
Processing: /content/drive/MyDrive/eu_ai_Act/2024.01.24-Decision-AI-Office.pdf
Successfully read: 2024.01.24-Decision-AI-Office.pdf
Processing: /content/drive/MyDrive/eu_ai_Act/2024.01.24-EuroHPC.pdf
Successfully read: 2024.01.24-EuroHPC.pdf
Documents successfully loaded: dict_keys(['EY_eu_ai_act.pdf', 'Official_eu_ai_act.pdf', 'KPMG_eu_ai_act.pdf', '2024.01.24-Communication-Start-Ups.pdf', '2024.01.24-Decision-AI-Office.pdf', '2024.01.24-EuroHPC.pdf'])


In [ ]:
print(documents['EY_eu_ai_act.pdf'][10])

['10                                   Back to contents    © 202 4 EYGM Limited. All Rights Reserved  \n  EYG no. 011630 -23Gbl  \n   • Predictive policing of individuals  based on the profiling or assessment of their personality traits and \ncharacteristics  to predict the risk of a natural person committing a criminal offen se. This prohibition does \nnot apply to AI systems used in assessment of involvement in occurred criminal activity  which is already \nbased on objective and verifiable facts . \n• Law enforcement  use of real -time remote biometric identification (RBI) systems in publicly accessible \nspaces (certain exceptions apply subject to prior judicial authorization and for strictly defined lists of \ncriminal offenses ). \nHigh-risk systems : which use cases are subject to conformity  \nassessments and  obligations?  \nThe AI Act identifies high-risk uses in Annex I and Annex III . The European Commission is empowered to update  \nthese annexes  as new uses  and risks ar

# Preprocessing

In [ ]:
def to_lowercase(text):
    return text.lower()


In [ ]:
import nltk  # Make sure to import nltk
from nltk.corpus import stopwords

# Download the stopwords dataset
nltk.download('stopwords')

# Create a set of English stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    tokens = text.split()
    return ' '.join([word for word in tokens if word not in stop_words])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def tokenize(text):
    return text.split()

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    tokens = text.split()
    return ' '.join([lemmatizer.lemmatize(token) for token in tokens])

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
import re

def remove_punctuation(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)

In [ ]:
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

In [ ]:
import contractions

def expand_contractions(text):
    return contractions.fix(text)

In [ ]:
def normalize_unicode(text):
    return text.encode('ascii', 'ignore').decode('ascii')

In [ ]:
def normalize_whitespace(text):
    return ' '.join(text.split())

In [ ]:
processed_documents = {}

for filename, content in documents.items():
    print(f"Preprocessing: {filename}")

    # Initialize a list to hold the page number and tokenized content
    pages_tokens = []

    for page_number, page_text in content.items():  # Loop through each page of the document

        # If page_text is a list, join it into a single string
        if isinstance(page_text, list):
            page_text = ' '.join(page_text)

        # Step 1: Expand contractions
        text_without_contractions = expand_contractions(page_text)

        # Step 2: Normalize Unicode
        text_normalized_unicode = normalize_unicode(text_without_contractions)

        # Step 3: Remove numbers (optional)
        text_without_numbers = remove_numbers(text_normalized_unicode)

        # Step 4: Lowercase text
        text_lowercased = to_lowercase(text_without_numbers)

        # Step 5: Remove punctuation
        text_without_punctuation = remove_punctuation(text_lowercased)

        # Step 6: Remove stop words
        text_without_stop_words = remove_stopwords(text_without_punctuation)

        # Step 7: Lemmatize text
        text_lemmatized = lemmatize(text_without_stop_words)

        # Step 8: Normalize whitespace
        text_normalized_whitespace = normalize_whitespace(text_lemmatized)

        # Step 9: Tokenize text
        text_tokenized = tokenize(text_normalized_whitespace)

        # Append the page number and tokenized content
        pages_tokens.append((page_number, text_tokenized))

    # Save the preprocessed document with (page number, list of tokens) pairs
    processed_documents[filename] = pages_tokens
    print(f"Preprocessed {filename}: {processed_documents[filename]}")

# Check the preprocessed documents
print("Documents successfully preprocessed:", processed_documents.keys())

Preprocessing: EY_eu_ai_act.pdf
Processing page: 1
Processing page: 2
Processing page: 3
Processing page: 4
Processing page: 5
Processing page: 6
Processing page: 7
Processing page: 8
Processing page: 9
Processing page: 10
Processing page: 11
Processing page: 12
Processing page: 13
Processing page: 14
Processing page: 15
Processing page: 16
Processing page: 17
Preprocessed EY_eu_ai_act.pdf: [(1, ['back', 'content', 'eygm', 'limited', 'right', 'reserved', 'eyg', 'gbl', 'european', 'union', 'artificial', 'intelligence', 'act', 'latest', 'development', 'key', 'takeaway', 'july']), (2, ['back', 'content', 'eygm', 'limited', 'right', 'reserved', 'eyg', 'gbl', 'eu', 'artificial', 'intelligence', 'act', 'updated', 'july', 'eu', 'ai', 'act', 'adopted', 'end', 'may', 'enter', 'force', 'august', 'thereafter', 'act', 'tiered', 'compliance', 'obligation', 'progressively', 'come', 'application', 'phased', 'timeline', 'requirement', 'taking', 'effect', 'mid', 'rtificial', 'intelligence', 'ai', 'act'

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the sentence-transformer model (replace with your preferred embedding model)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Initialize dictionary to store document embeddings
document_embeddings = {}

# Iterate over the preprocessed documents
for filename, pages in processed_documents.items():
    print(f"Generating embeddings for: {filename}")

    page_embeddings = []  # To store embeddings for each page

    for page_number, tokens in pages:
        # Convert the list of tokens back into a single string (or keep as a list if needed for your model)
        page_text = ' '.join(tokens)

        # Generate the embedding for the page content
        embedding = model.encode(page_text)

        # Store the embedding alongside the page number
        page_embeddings.append((page_number, embedding))

    # Save the embeddings with the document name as the key
    document_embeddings[filename] = page_embeddings

# Check the generated embeddings
print("Embeddings generated for documents:", document_embeddings.keys())

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generating embeddings for: EY_eu_ai_act.pdf
Generating embeddings for: Official_eu_ai_act.pdf
Generating embeddings for: KPMG_eu_ai_act.pdf
Generating embeddings for: 2024.01.24-Communication-Start-Ups.pdf
Generating embeddings for: 2024.01.24-Decision-AI-Office.pdf
Generating embeddings for: 2024.01.24-EuroHPC.pdf
Embeddings generated for documents: dict_keys(['EY_eu_ai_act.pdf', 'Official_eu_ai_act.pdf', 'KPMG_eu_ai_act.pdf', '2024.01.24-Communication-Start-Ups.pdf', '2024.01.24-Decision-AI-Office.pdf', '2024.01.24-EuroHPC.pdf'])


In [ ]:
import chromadb
from chromadb.config import Settings

client = chromadb.Client()

collection = client.create_collection(name="document_embeddings")

for filename, pages in document_embeddings.items():
    for page_number, embedding in pages:
        collection.add(
            documents=[filename],  # Document name
            embeddings=[embedding],  # Corresponding embedding
            metadatas=[{"page_number": page_number}],  # Optional metadata
            ids=[f"{filename}_page_{page_number}"]  # Unique ID for each embedding
        )

all_documents = collection.get()
print("All documents in the collection:", all_documents)

All documents in the collection: {'ids': ['EY_eu_ai_act.pdf_page_1', 'EY_eu_ai_act.pdf_page_2', 'EY_eu_ai_act.pdf_page_3', 'EY_eu_ai_act.pdf_page_4', 'EY_eu_ai_act.pdf_page_5', 'EY_eu_ai_act.pdf_page_6', 'EY_eu_ai_act.pdf_page_7', 'EY_eu_ai_act.pdf_page_8', 'EY_eu_ai_act.pdf_page_9', 'EY_eu_ai_act.pdf_page_10', 'EY_eu_ai_act.pdf_page_11', 'EY_eu_ai_act.pdf_page_12', 'EY_eu_ai_act.pdf_page_13', 'EY_eu_ai_act.pdf_page_14', 'EY_eu_ai_act.pdf_page_15', 'EY_eu_ai_act.pdf_page_16', 'EY_eu_ai_act.pdf_page_17', 'Official_eu_ai_act.pdf_page_1', 'Official_eu_ai_act.pdf_page_2', 'Official_eu_ai_act.pdf_page_3', 'Official_eu_ai_act.pdf_page_4', 'Official_eu_ai_act.pdf_page_5', 'Official_eu_ai_act.pdf_page_6', 'Official_eu_ai_act.pdf_page_7', 'Official_eu_ai_act.pdf_page_8', 'Official_eu_ai_act.pdf_page_9', 'Official_eu_ai_act.pdf_page_10', 'Official_eu_ai_act.pdf_page_11', 'Official_eu_ai_act.pdf_page_12', 'Official_eu_ai_act.pdf_page_13', 'Official_eu_ai_act.pdf_page_14', 'Official_eu_ai_act.pdf_

In [19]:
total_documents = collection.count()
print(total_documents)

539
